# Sequential Model Demo Full CNN

## 00. Imports

In [ ]:
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist
from sklearn.metrics import confusion_matrix

In [ ]:
import sys
sys.path.append("../")

In [ ]:
from src.activation.relu import ReluLayer
from src.layers.pooling import MaxPoolLayer
from src.activation.softmax import SoftmaxLayer
from src.layers.dense import DenseLayer
from src.layers.flatten import FlattenLayer
from src.layers.convolutional import ConvLayer2D, SuperFastConvLayer2D
from src.layers.dropout import DropoutLayer
from src.model.sequential import SequentialModel
from src.utils.core import convert_categorical2one_hot, convert_prob2categorical
from src.utils.metrics import softmax_accuracy
from src.optimizers.gradient_descent import GradientDescent
from src.optimizers.rms_prop import RMSProp
from src.optimizers.adam import Adam
from src.utils.plots import lines

In [ ]:
x_train = np.load('./covid_19_data2/x.npy')
x_train = x_train.reshape(len(x_train), 224,224,1)
y_train = np.load('./covid_19_data2/y.npy')


In [ ]:

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
print(x_train.shape,y_train.shape)

In [ ]:
x_train=x_train[:1000]
y_train=y_train[:1000]
print(x_train.shape,y_train.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_train,
                                                    y_train, 
                                                    test_size=0.3, 
                                                    shuffle=True, 
                                                    random_state=1004)


## 03. Build model

In [ ]:
layers = [
    # input (N, 28, 28, 1) out (N, 28, 28, 32)
    SuperFastConvLayer2D.initialize(filters=64, kernel_shape=(3, 3, 1), stride=1, padding="same"),
    # input (N, 28, 28, 32) out (N, 28, 28, 32)
    ReluLayer(),
    MaxPoolLayer(pool_size=(2, 2), stride=2),
    DropoutLayer(keep_prob=0.75),
    # input (N, 28, 28, 32) out (N, 28, 28, 32)
    SuperFastConvLayer2D.initialize(filters=128, kernel_shape=(3, 3, 64), stride=1, padding="same"),
    # input (N, 28, 28, 32) out (N, 28, 28, 32)
    ReluLayer(),
    # input (N, 28, 28, 32) out (N, 14, 14, 32)
    MaxPoolLayer(pool_size=(2, 2), stride=2),
    DropoutLayer(keep_prob=0.75),
    # input (N, 14, 14, 32) out (N, 14, 14, 32)
    # input (N, 7, 7, 64) out (N, 7 * 7 * 64)
    FlattenLayer(),
    # input (N, 7 * 7 * 64) out (N, 256)
    DenseLayer.initialize(units_prev=56*56*128, units_curr=3),
    # input (N, 256) out (N, 256)
    # input (N, 10) out (N, 10)
    SoftmaxLayer()
]

optimizer = Adam(lr=0.003)

model = SequentialModel(
    layers=layers,
    optimizer=optimizer
)

## 04. Train

In [ ]:
model.train(
    x_train=X_train, 
    y_train=y_train, 
    x_test=X_test, 
    y_test=y_test, 
    epochs=5,
    bs=4,
    verbose=True
)

## 05. Predict and examine results

In [ ]:
lines(
    y_1=np.array(model.history["train_acc"]),
    y_2=np.array(model.history["test_acc"]),
    label_1="train",
    label_2="test",
    title="ACCURACY",
    fig_size=(16,10),
    path="../viz/cnn_acc.png"
)

In [ ]:
lines(
    y_1=np.array(model.history["train_loss"]),
    y_2=np.array(model.history["test_loss"]),
    label_1="train",
    label_2="test",
    title="LOSS",
    fig_size=(16,10),
    path="../viz/cnn_loss.png"
)

In [ ]:
y_hat = model.predict(X_valid)
acc = softmax_accuracy(y_hat, y_valid)
print("acc: ", acc)

In [ ]:
y_hat = convert_prob2categorical(y_hat)
y_valid = convert_prob2categorical(y_valid)

In [ ]:
df_cm = pd.DataFrame(
    confusion_matrix(y_valid, y_hat), 
    range(10), 
    range(10)
)
plt.figure(figsize = (16,16))
sn.heatmap(df_cm, annot=True, cmap="YlGnBu", linewidths=.5, cbar=False)
plt.savefig("../viz/cm.png", dpi=100)
plt.show()